In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger
from tensorflow.keras.metrics import BinaryIoU, Precision, Recall

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
TRAIN_IMAGE_DIR = "/content/drive/MyDrive/UMich/PR/EuroPalletSeg/train/images"
TRAIN_MASK_DIR = "/content/drive/MyDrive/UMich/PR/EuroPalletSeg/train/masks"
VAL_IMAGE_DIR   = "/content/drive/MyDrive/UMich/PR/EuroPalletSeg/valid/images"
VAL_MASK_DIR    = "/content/drive/MyDrive/UMich/PR/EuroPalletSeg/valid/masks"
IMG_SIZE = (256, 256)
BATCH_SIZE = 32

In [ ]:

def process_image_mask(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE) / 255.0

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, IMG_SIZE, method="nearest")
    mask = tf.cast(mask > 127, tf.uint8)  # 0 or 1 only

    return image, mask

def load_dataset(image_dir, mask_dir):
    image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir)])
    mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir)])
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    dataset = dataset.map(process_image_mask, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [ ]:
train_ds = load_dataset(TRAIN_IMAGE_DIR, TRAIN_MASK_DIR).shuffle(200).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = load_dataset(VAL_IMAGE_DIR, VAL_MASK_DIR).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
def build_unet_with_efficientnetb3(input_shape=(256, 256, 3)):
    base_model = EfficientNetB3(input_shape=input_shape, include_top=False, weights='imagenet')

    skips = [
        base_model.get_layer("block1a_project_bn").output,
        base_model.get_layer("block2a_expand_activation").output,
        base_model.get_layer("block3a_expand_activation").output,
        base_model.get_layer("block4a_expand_activation").output,
        base_model.get_layer("block6a_expand_activation").output,
    ]

    encoder_output = base_model.get_layer("top_activation").output
    x = encoder_output

    def decoder_block(x, skip, filters):
        x = layers.Conv2DTranspose(filters, 3, strides=2, padding='same')(x)

        if x.shape[1] != skip.shape[1] or x.shape[2] != skip.shape[2]:
            skip = layers.UpSampling2D(size=(x.shape[1] // skip.shape[1], x.shape[2] // skip.shape[2]),
                                       interpolation='bilinear')(skip)

        x = layers.Concatenate()([x, skip])
        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Dropout(0.3)(x)
        return x

    x = decoder_block(x, skips[4], 240)
    x = decoder_block(x, skips[3], 120)
    x = decoder_block(x, skips[2], 72)
    x = decoder_block(x, skips[1], 48)
    x = decoder_block(x, skips[0], 32)

    output = layers.Conv2D(1, 1, padding='same', activation='sigmoid')(x)

    return models.Model(inputs=base_model.input, outputs=output)

In [ ]:
log_dir = "/content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/logs"

callbacks = [
    ModelCheckpoint(
        filepath="/content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/models/best_model_epoch{epoch:02d}_valacc{val_accuracy:.2f}.h5",
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1,
        mode="max"
    ),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1, min_lr=1e-6),
    TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True,
        write_images=True
    ),
    CSVLogger(
        "/content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/training_log.csv",
        append=True
    )
]

In [ ]:
with tf.device('/GPU:0'):
    model = build_unet_with_efficientnetb3()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[
            'accuracy',
            tf.keras.metrics.BinaryIoU(target_class_ids=[1], threshold=0.5),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ]
    )

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=100,
        callbacks=callbacks
    )

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7199 - binary_io_u: 0.4012 - loss: 0.5429 - precision: 0.4482 - recall: 0.7707
Epoch 1: val_accuracy improved from -inf to 0.72068, saving model to /content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/models/best_model_epoch01_valacc0.72.h5


94/94 ━━━━━━━━━━━━━━━━━━━━ 670s 4s/step - accuracy: 0.7212 - binary_io_u: 0.4031 - loss: 0.5411 - precision: 0.4501 - recall: 0.7718 - val_accuracy: 0.7207 - val_binary_io_u: 0.0000e+00 - val_loss: 0.9109 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9763 - binary_io_u: 0.9025 - loss: 0.1543 - precision: 0.9401 - recall: 0.9575
Epoch 2: val_accuracy improved from 0.72068 to 0.77033, saving model to /content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/models/best_model_epoch02_valacc0.77.h5


94/94 ━━━━━━━━━━━━━━━━━━━━ 27s 281ms/step - accuracy: 0.9763 - binary_io_u: 0.9027 - loss: 0.1540 - precision: 0.9403 - recall: 0.9576 - val_accuracy: 0.7703 - val_binary_io_u: 0.1893 - val_loss: 0.5280 - val_precision: 0.9309 - val_recall: 0.1920 - learning_rate: 1.0000e-04
Epoch 3/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.9847 - binary_io_u: 0.9357 - loss: 0.1097 - precision: 0.9672 - recall: 0.9664
Epoch 3: val_accuracy improved from 0.77033 to 0.89894, saving model to /content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/models/best_model_epoch03_valacc0.90.h5


94/94 ━━━━━━━━━━━━━━━━━━━━ 27s 282ms/step - accuracy: 0.9847 - binary_io_u: 0.9358 - loss: 0.1096 - precision: 0.9672 - recall: 0.9665 - val_accuracy: 0.8989 - val_binary_io_u: 0.7142 - val_loss: 0.2800 - val_precision: 0.7728 - val_recall: 0.9040 - learning_rate: 1.0000e-04
Epoch 4/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9865 - binary_io_u: 0.9425 - loss: 0.0958 - precision: 0.9700 - recall: 0.9708
Epoch 4: val_accuracy did not improve from 0.89894
94/94 ━━━━━━━━━━━━━━━━━━━━ 24s 255ms/step - accuracy: 0.9865 - binary_io_u: 0.9425 - loss: 0.0957 - precision: 0.9701 - recall: 0.9708 - val_accuracy: 0.8653 - val_binary_io_u: 0.6518 - val_loss: 0.4060 - val_precision: 0.7009 - val_recall: 0.9028 - learning_rate: 1.0000e-04
Epoch 5/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9878 - binary_io_u: 0.9476 - loss: 0.0858 - precision: 0.9740 - recall: 0.9722
Epoch 5: val_accuracy did not improve from 0.89894
94/94 ━━━━━━━━━━━━━━━━━━━━ 24s 255ms/step - accuracy: 

94/94 ━━━━━━━━━━━━━━━━━━━━ 26s 276ms/step - accuracy: 0.9890 - binary_io_u: 0.9531 - loss: 0.0726 - precision: 0.9755 - recall: 0.9764 - val_accuracy: 0.9272 - val_binary_io_u: 0.7898 - val_loss: 0.2810 - val_precision: 0.8035 - val_recall: 0.9789 - learning_rate: 1.0000e-04
Epoch 8/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9898 - binary_io_u: 0.9562 - loss: 0.0669 - precision: 0.9767 - recall: 0.9785
Epoch 8: val_accuracy improved from 0.92724 to 0.96656, saving model to /content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/models/best_model_epoch08_valacc0.97.h5


...

94/94 ━━━━━━━━━━━━━━━━━━━━ 26s 274ms/step - accuracy: 0.9936 - binary_io_u: 0.9724 - loss: 0.0314 - precision: 0.9838 - recall: 0.9882 - val_accuracy: 0.9877 - val_binary_io_u: 0.9569 - val_loss: 0.0500 - val_precision: 0.9808 - val_recall: 0.9752 - learning_rate: 2.5000e-05
Epoch 27/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9937 - binary_io_u: 0.9729 - loss: 0.0309 - precision: 0.9839 - recall: 0.9886
Epoch 27: val_accuracy improved from 0.98774 to 0.98774, saving model to /content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/models/best_model_epoch27_valacc0.99.h5


94/94 ━━━━━━━━━━━━━━━━━━━━ 27s 281ms/step - accuracy: 0.9937 - binary_io_u: 0.9729 - loss: 0.0308 - precision: 0.9839 - recall: 0.9886 - val_accuracy: 0.9877 - val_binary_io_u: 0.9570 - val_loss: 0.0491 - val_precision: 0.9803 - val_recall: 0.9757 - learning_rate: 2.5000e-05
Epoch 28/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.9936 - binary_io_u: 0.9725 - loss: 0.0306 - precision: 0.9837 - recall: 0.9884
Epoch 28: val_accuracy improved from 0.98774 to 0.98787, saving model to /content/drive/MyDrive/UMich/PR/EuroPalletSeg/results/models/best_model_epoch28_valacc0.99.h5


94/94 ━━━━━━━━━━━━━━━━━━━━ 26s 273ms/step - accuracy: 0.9936 - binary_io_u: 0.9725 - loss: 0.0306 - precision: 0.9837 - recall: 0.9884 - val_accuracy: 0.9879 - val_binary_io_u: 0.9573 - val_loss: 0.0473 - val_precision: 0.9828 - val_recall: 0.9736 - learning_rate: 2.5000e-05
Epoch 29/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9938 - binary_io_u: 0.9734 - loss: 0.0298 - precision: 0.9838 - recall: 0.9893
Epoch 29: val_accuracy did not improve from 0.98787
94/94 ━━━━━━━━━━━━━━━━━━━━ 24s 252ms/step - accuracy: 0.9938 - binary_io_u: 0.9734 - loss: 0.0298 - precision: 0.9838 - recall: 0.9893 - val_accuracy: 0.9875 - val_binary_io_u: 0.9561 - val_loss: 0.0485 - val_precision: 0.9818 - val_recall: 0.9733 - learning_rate: 2.5000e-05
Epoch 30/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9938 - binary_io_u: 0.9734 - loss: 0.0296 - precision: 0.9843 - recall: 0.9888
Epoch 30: val_accuracy did not improve from 0.98787
94/94 ━━━━━━━━━━━━━━━━━━━━ 25s 253ms/step - accura

94/94 ━━━━━━━━━━━━━━━━━━━━ 26s 277ms/step - accuracy: 0.9939 - binary_io_u: 0.9738 - loss: 0.0290 - precision: 0.9840 - recall: 0.9895 - val_accuracy: 0.9881 - val_binary_io_u: 0.9583 - val_loss: 0.0473 - val_precision: 0.9806 - val_recall: 0.9769 - learning_rate: 2.5000e-05
Epoch 32/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.9940 - binary_io_u: 0.9740 - loss: 0.0284 - precision: 0.9843 - recall: 0.9894
Epoch 32: val_accuracy did not improve from 0.98814
94/94 ━━━━━━━━━━━━━━━━━━━━ 25s 256ms/step - accuracy: 0.9940 - binary_io_u: 0.9740 - loss: 0.0284 - precision: 0.9843 - recall: 0.9894 - val_accuracy: 0.9881 - val_binary_io_u: 0.9581 - val_loss: 0.0473 - val_precision: 0.9818 - val_recall: 0.9754 - learning_rate: 2.5000e-05
Epoch 33/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.9940 - binary_io_u: 0.9739 - loss: 0.0282 - precision: 0.9844 - recall: 0.9891
Epoch 33: val_accuracy did not improve from 0.98814

Epoch 33: ReduceLROnPlateau reducing learning rat

94/94 ━━━━━━━━━━━━━━━━━━━━ 26s 274ms/step - accuracy: 0.9944 - binary_io_u: 0.9759 - loss: 0.0248 - precision: 0.9860 - recall: 0.9896 - val_accuracy: 0.9883 - val_binary_io_u: 0.9590 - val_loss: 0.0455 - val_precision: 0.9811 - val_recall: 0.9771 - learning_rate: 3.1250e-06
Epoch 53/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.9944 - binary_io_u: 0.9757 - loss: 0.0249 - precision: 0.9861 - recall: 0.9893
Epoch 53: val_accuracy did not improve from 0.98833
94/94 ━━━━━━━━━━━━━━━━━━━━ 24s 251ms/step - accuracy: 0.9944 - binary_io_u: 0.9757 - loss: 0.0249 - precision: 0.9861 - recall: 0.9893 - val_accuracy: 0.9883 - val_binary_io_u: 0.9588 - val_loss: 0.0456 - val_precision: 0.9809 - val_recall: 0.9771 - learning_rate: 3.1250e-06
Epoch 54/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.9944 - binary_io_u: 0.9758 - loss: 0.0248 - precision: 0.9862 - recall: 0.9893
Epoch 54: val_accuracy did not improve from 0.98833
94/94 ━━━━━━━━━━━━━━━━━━━━ 24s 252ms/step - accura